In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
import os
import seaborn as sns
from functools import partial
sns.set_context('notebook')
import matplotlib.pyplot as plt
import cv2


In [2]:
def hist_equal(img):
    '''takes image and an output space [0,L] as an input and gives an equalized image(in float) as output'''
    if len(img.shape) != 2:
        R, G, B = cv2.split(img)
        output1_R = cv2.equalizeHist(R)
        output1_G = cv2.equalizeHist(G)
        output1_B = cv2.equalizeHist(B)
        equ = cv2.merge((output1_R, output1_G, output1_B))
        return equ

#### Load Directories and get file paths

In [3]:
my_dir = os.getcwd()  # grabs current work dir
all_file_paths = os.listdir()  # grabs all items in current work dir

data_path = [data for data in all_file_paths if 'traindata' in data.lower()]  # searches for data folder and retrieves

picture_folder_path = os.path.join(my_dir, data_path[0])  # joins data folder and current work dir to get picture path
picture_folder_path

all_pictures = os.listdir(picture_folder_path)  # list of pictures names in folder

picture_complete_path = [os.path.join(picture_folder_path, pic) for pic in
                         all_pictures]  # complete path to each picture

PP_path = os.path.join(my_dir, "TrainData_PP")  # PP: Post Processed



# split the picture path and return only the OG file name
file_names = [picture_complete_path[x].split('TrainData\\')[1] for x in range(len(picture_complete_path))]

#### Test Files, Load Test Images

In [ ]:
# img_1 = load_img(picture_complete_path[0])
# print("FIRST IMAGE DETAILS:", "\nClass: ", type(img_1), "\nFile Format: ", img_1.format,
#       "\nImage type: ", img_1.mode, '\nImage Size:', img_1.size)
# print()

# imag_1_open_cv = cv2.imread(picture_complete_path[0])  # read image using cv
# test_hist_img1 = hist_equal(imag_1_open_cv, 1)  # use equilizer function

# cv2.imshow('No_PostProcessing', imag_1_open_cv)
# cv2.imshow('Post_Processing', test_hist_img1)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


#### Run histogram equilizer function and write images to post processing folder

In [ ]:

if len(os.listdir(PP_path)) == 0:  # ONLY RUN IF THE FOLDER IS EMPTY!!
    post_processed_files = []

    for n, img in enumerate(picture_complete_path):
        load_temp_file = cv2.imread(img)
        post_processed_files.append(hist_equal(load_temp_file, 1))

    #     cv2.imshow('Post_Processing', post_processed_files[n])
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

        # take [0 ,1]  -> scale to 255
        post_processed_files[n] = cv2.convertScaleAbs(post_processed_files[n], alpha=(255.0))
        cv2.imwrite(os.path.join(PP_path, file_names[n]), post_processed_files[n],[int(cv2.IMWRITE_JPEG_QUALITY), 100])




## Defines how may augmentation rounds to generate photos

In [4]:
#  create PP image file paths but strip .j
pp_image_path = [os.path.join(PP_path, file_names[n]) for n in range(len(file_names))]
pp_image_prexfix = [os.path.join(PP_path, file_names[n]).strip('.jpg')[0] for n in range(len(file_names))]
aug_rounds = 2
aug_names = [f'augmen_round{i+1}_' for i in range(2)]
aug_names

['augmen_round1_', 'augmen_round2_']

## Runs Agumentation / ImageDataGenerator

In [5]:
# create data generator: Gives parameters to augment 
num_orig_files = len(picture_complete_path)
print(num_orig_files)

datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.1,
                             zoom_range=0.25,
                             horizontal_flip=True,
                             rotation_range=25,
                             fill_mode ='reflect' )

pregen_imgs = [cv2.imread(pp_image_path[i]) for i in range(num_orig_files)]  # list of images
pregen_imgs = np.asarray(pregen_imgs) # images list into array list
# np.expand_dims(test_imgs,0)

it = datagen.flow(pregen_imgs, batch_size=1025, shuffle=False,)

for i in range(aug_rounds):
    batch = it.next()
    for k in range(batch.shape[0]):
        image = cv2.convertScaleAbs(batch[k], alpha=(255.0))
#         cv2.imshow('test', image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        cv2.imwrite(os.path.join(PP_path, aug_names[i] + file_names[k]), image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

#     it = datagen.flow(pregen_imgs[0:10] ,batch_size=1, shuffle=False, save_to_dir=PP_path, save_prefix=f'aug_names{i}',)
    

1025


In [9]:
del pregen_imgs, it, batch  # restore memory